<a href="https://colab.research.google.com/github/khickey25/nlp_shap_notebooks/blob/master/8_26_2020_Data_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Prep for Fake news

I needed to fix the text before it would be processed and saved to the pickle file, so I do that here.

In [ ]:


#housekeeping steps
from google.colab import drive
drive.mount('/content/drive')

!pip install shap

import pandas as pd
%tensorflow_version 1.13.1
import tensorflow as tf
from tensorflow.keras.preprocessing import text
import keras.backend.tensorflow_backend as K
K.set_session
import matplotlib.pyplot as plt
import shap
import sklearn.metrics as skm
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import pprint
import re

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.13.1`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
csv_file = "/content/drive/My Drive/test_data/fake_news_20200823.csv"
pickle_file = "/content/drive/My Drive/test_data/fake-news-20200824-v2.pickle"
full_df = pd.read_csv(csv_file).reset_index(drop=True)
DF = pd.read_csv(csv_file)
with open(pickle_file, 'rb') as pickle_file:
  text_train = pickle.load(pickle_file)
  y_train = pickle.load(pickle_file)
  text_test = pickle.load(pickle_file)
  y_test = pickle.load(pickle_file)
for df in [text_train, y_train, text_test, y_test]:
  df.reset_index(drop=True, inplace=True)

#scratch debugging info: not info here
a,b,c,d =  train_test_split(full_df.text, full_df.label, test_size=0.2, 
                                                    random_state=0)

In [ ]:
#hacky way to remove reuters from original text and 
#update the content column so that it does not have any
#"(Reuters) -" in there. 
#fake articles tend to have a sourcing sentence at end of article,
#so use crude implementation to remove last 40 characters of text
#takes about 90sec to run
for i, text in enumerate(DF['text']):
    if '(Reuters) -' in text:
      c = text[re.search("\) -", text).end()+1:]
      DF.loc[i,'text'] = c
    else:
      DF.loc[i, 'text'] = text[:-40]


DF['content'] = DF['title'] + DF['text']


In [ ]:
DF.iloc[0]['content']

'No. 2 Senate Republican: health bill to be discussed Tuesday before voteSenator John Cornyn, the No. 2 Senate Republican, said on Monday that various options for a healthcare bill will be discussed at a policy lunch on Tuesday prior to the first procedural vote. Republican Senator Orrin Hatch said he did not expect Senator John McCain, who is recuperating from surgery in his home state of Arizona, to return to Washington in time for Tuesday’s healthcare vote. '

# Data Prep

Same preprocessing steps as done in Tao's notebook.

In [ ]:
#@title
#code for TODO 8/26/12:28
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings
#data = sent_df1
if not sys.warnoptions:
    warnings.simplefilter("ignore")
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six',
                   'seven','eight','nine','ten','may','also','across','among','beside',
                   'however','yet','within', 'almost'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#@title

review_text = DF.content.str.lower()

review_text = review_text.apply(cleanHtml)
review_text = review_text.apply(cleanPunc)
review_text = review_text.apply(keepAlpha)

review_text = review_text.apply(removeStopWords)
review_text = review_text.apply(stemming)

In [ ]:
#@title
from sklearn.model_selection import train_test_split
text_train, text_test, y_train, y_test = train_test_split(review_text, DF.label, test_size=0.2, 
                                                    random_state=0)

In [ ]:
#@title
import pickle

#saving it to my google drive
file_path = '/content/drive/My Drive/test_data/fake-news-20200826-cleaned.pickle'
with open(file_path, 'wb') as pickle_file:
    pickle.dump(text_train, pickle_file)
    pickle.dump(y_train, pickle_file)
    pickle.dump(text_test, pickle_file)
    pickle.dump(y_test, pickle_file)